In [351]:
import requests as rq
import json
import pandas as pd
pd.options.plotting.backend = "plotly"
pd.set_option('display.precision', 4,
              'display.colheader_justify', 'center')
import numpy as np
import warnings
import pytz
import datetime
import time
from IPython.display import clear_output

In [2]:
# Get demo API key
def get_demo_key():
    f = open("/home/vikas/Documents/CG_demo_key.json")
    key_dict = json.load(f)
    return key_dict["key"] 

In [7]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : get_demo_key() 
}

In [3]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"] 

## Get list of exchanges

In [4]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [5]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")    

In [9]:
# Valid values for results per page is between 1-250
exchange_params = {
            "per_page": 250,
            "page": 1
}

In [10]:
exchange_list_response = get_response("/exchanges", use_demo, exchange_params, PUB_URL)

In [11]:
df_ex = pd.DataFrame(exchange_list_response)

In [12]:
df_ex

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,bybit_spot,Bybit,2018.0,British Virgin Islands,Bybit is a cryptocurrency exchange that offers...,https://www.bybit.com,https://coin-images.coingecko.com/markets/imag...,False,10.0,1.0,32158.188622,22423.270927
1,gdax,Coinbase Exchange,2012.0,United States,,https://www.coinbase.com/,https://coin-images.coingecko.com/markets/imag...,False,10.0,2.0,14825.942967,14825.942967
2,okex,OKX,2017.0,Seychelles,,https://www.okx.com,https://coin-images.coingecko.com/markets/imag...,False,10.0,3.0,14619.888740,14619.888740
3,gate,Gate.io,NaN,Cayman Islands,"Gate was established in 2013, and it is the to...",https://www.gate.io/,https://coin-images.coingecko.com/markets/imag...,False,10.0,4.0,17907.743194,11861.251440
4,kucoin,KuCoin,2017.0,Seychelles,"Launched in September 2017, KuCoin is a global...",https://www.kucoin.com/,https://coin-images.coingecko.com/markets/imag...,False,10.0,5.0,5974.994369,4433.619730
...,...,...,...,...,...,...,...,...,...,...,...,...
245,curve-bsc,Curve (BSC),NaN,None,,https://curve.fi/#/bsc/swap,https://coin-images.coingecko.com/markets/imag...,False,NaN,244.0,2.417181,2.417181
246,bunnyswap,BunnySwap,NaN,None,,,https://coin-images.coingecko.com/markets/imag...,False,5.0,245.0,1.990213,1.990213
247,spartadex,SpartaDEX,2023.0,Estonia,,https://app.spartadex.io/dex,https://coin-images.coingecko.com/markets/imag...,False,5.0,246.0,2.067631,2.067631
248,pearl-exchange,PearlFi,NaN,None,,https://www.pearl.exchange/swap,https://coin-images.coingecko.com/markets/imag...,False,5.0,247.0,1.932746,1.932746


In [13]:
list(df_ex.columns)

['id',
 'name',
 'year_established',
 'country',
 'description',
 'url',
 'image',
 'has_trading_incentive',
 'trust_score',
 'trust_score_rank',
 'trade_volume_24h_btc',
 'trade_volume_24h_btc_normalized']

#### List exchanges by trading volume

In [58]:
df_ex_subset = df_ex[["id", "name", "country", "trade_volume_24h_btc"]]
df_ex_subset = df_ex_subset.sort_values(by = ["trade_volume_24h_btc"], ascending = False)

In [66]:
df_ex_subset["country"].unique()

array(['Cayman Islands', 'Seychelles', 'British Virgin Islands',
       'United States', 'Lithuania', 'Singapore', 'Malta', 'South Korea',
       'Canada', None, 'United Arab Emirates', 'Gibraltar', 'Australia',
       'Bermuda', 'Hong Kong', 'Spain', 'Turkey', 'Netherlands',
       'Bahamas', 'Luxembourg', 'Saint Vincent and the Grenadines',
       'United Kingdom', 'Vanuatu', 'Belize', 'Argentina', 'Costa Rica',
       'Samoa', 'Estonia', 'Thailand', 'Japan', 'Indonesia', 'Taiwan',
       'Philippines', 'Switzerland', 'Brazil', 'India', 'South Africa',
       'Unknown or Invalid Region', 'Liechtenstein', 'Panama', 'Georgia',
       'Jamaica', 'Slovenia'], dtype=object)

In [72]:
df_ex_subset[(df_ex_subset["country"] == "Hong Kong")]

,id,name,country,trade_volume_24h_btc
12,hashkey_exchange,HashKey Exchange,Hong Kong,1830.439435
113,tokpie,Tokpie,Hong Kong,75.880736


## Get exchange tickers

In [218]:
def get_trade_exchange(id, base_curr, target_curr):
    
    exchange_ticker_response = get_response(f"/exchanges/{id}/tickers",
                                            use_demo,
                                            {},
                                            PUB_URL)
    
    found_match = ""
    
    for ticker in exchange_ticker_response["tickers"]:
        if ticker["base"] == base_curr and ticker["target"] == target_curr:
            found_match = ticker
            break
            
    if found_match == "":
        warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
    
    return found_match

In [230]:
test_data = get_trade_exchange("gdax", "ETH", "USD")

In [231]:
test_data

{'base': 'ETH',
 'target': 'USD',
 'market': {'name': 'Coinbase Exchange',
  'identifier': 'gdax',
  'has_trading_incentive': False},
 'last': 3390.93,
 'volume': 71026.18655201,
 'converted_last': {'btc': 0.05561415, 'eth': 1.00071, 'usd': 3390.93},
 'converted_volume': {'btc': 3950, 'eth': 71077, 'usd': 240844827},
 'trust_score': 'green',
 'bid_ask_spread_percentage': 0.01118,
 'timestamp': '2024-06-29T13:32:30+00:00',
 'last_traded_at': '2024-06-29T13:32:30+00:00',
 'last_fetch_at': '2024-06-29T13:33:35+00:00',
 'is_anomaly': False,
 'is_stale': False,
 'trade_url': 'https://www.coinbase.com/advanced-trade/spot/ETH-USD',
 'token_info_url': None,
 'coin_id': 'ethereum'}

## Convert to local timezone

In [107]:
"UTC" in pytz.all_timezones

True

In [318]:
def convert_to_local_tz(old_ts):
    
    new_tz = pytz.timezone("Europe/Amsterdam")
    old_tz = pytz.timezone("UTC")
    
    format = "%Y-%m-%dT%H:%M:%S+00:00"
    datetime_obj = datetime.datetime.strptime(old_ts, format)
    
    localized_ts = old_tz.localize(datetime_obj)
    new_ts = localized_ts.astimezone(new_tz)
    
    return new_ts   

## Collect data for multiple exchanges based on country

In [319]:
def get_trade_exchange_per_country(country,
                                   base_curr,
                                   target_curr):
    
    df_all = df_ex_subset[(df_ex_subset["country"] == country)]    
    
    exchanges_list = df_all["id"]
    ex_all = []    
       
    for exchange_id in exchanges_list:
        found_match = get_trade_exchange(exchange_id, base_curr, target_curr)
        if found_match == "":
            continue
        else:
            temp_dict = dict(
                             exchange = exchange_id,
                             last_price = found_match["last"],
                             last_vol   = found_match["volume"],
                             spread     = found_match["bid_ask_spread_percentage"],
                             trade_time = convert_to_local_tz(found_match["last_traded_at"])
                             )
            ex_all.append(temp_dict)
            
    return pd.DataFrame(ex_all)   

In [362]:
df_ex_all = get_trade_exchange_per_country("United States", "ETH", "USD")
df_ex_all

/tmp/ipykernel_25321/3109473306.py:16: UserWarning: No data found for ETH-USD pair in tapbit
  warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
/tmp/ipykernel_25321/3109473306.py:16: UserWarning: No data found for ETH-USD pair in binance_us
  warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
/tmp/ipykernel_25321/3109473306.py:16: UserWarning: No data found for ETH-USD pair in metalx
  warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")


,exchange,last_price,last_vol,spread,trade_time
0,gdax,3387.33,26041.0224,0.0103,2024-06-30 11:14:12+02:00
1,kraken,3387.95,2201.6060,0.0103,2024-06-30 11:10:41+02:00
2,gemini,3386.86,356.7442,0.0779,2024-06-30 11:08:22+02:00
3,itbit,3387.85,631.3139,0.0115,2024-06-30 11:11:49+02:00
4,coinlist,3378.75,0.3097,0.1233,2024-06-30 11:12:04+02:00


## Get exchange rates

In [386]:
def get_exchange_rate(base_curr):
    
    # This returns current BTC to base_curr exchange rate    
    exchange_rate_response = get_response(f"/exchange_rates",
                                          use_demo,
                                          {},
                                          PUB_URL)
    rate = ""
    try:
        rate = exchange_rate_response["rates"][base_curr.lower()]["value"]
    except KeyError as ke:
        print("Currency not found in the exchange rate API response:", ke)
        
    return rate    

In [400]:
get_exchange_rate("ETH")

18.144

## Get historical volume data

In [390]:
def get_vol_exchange(id, days, base_curr):
    
    vol_params = {"days": days}
    
    exchange_vol_response = get_response(f"/exchanges/{id}/volume_chart",
                                         use_demo,
                                         vol_params,
                                         PUB_URL)
    
    time, volume = [], []
    
    # Get exchange rate when base_curr is not BTC
    ex_rate = 1.0
    if base_curr != "BTC":
        ex_rate = get_exchange_rate(base_curr)
        
        # Give a warning when exchange rate is not found
        if ex_rate == "":
            print(f"Unable to find exchange rate for {base_curr}, vol will be reported in BTC")
            ex_rate = 1.0
    
    for i in range(len(exchange_vol_response)):
        # Convert to seconds
        s = exchange_vol_response[i][0] / 1000
        time.append(datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d'))
        
        # Default unit for volume is BTC
        volume.append(float(exchange_vol_response[i][1]) * ex_rate)
                      
    df_vol = pd.DataFrame(list(zip(time, volume)), columns = ["date", "volume"])
    
    # Calculate SMA for a specific window
    df_vol["volume_SMA"] = df_vol["volume"].rolling(7).mean()
    
    return df_vol.sort_values(by = ["date"], ascending = False).reset_index(drop = True)

## Collect and display aggregate per exchange

In [402]:
def display_agg_per_exchange(df_ex_all, base_curr):
    
    # Group data and calculate statistics per exchange    
    df_agg = (
        df_ex_all.groupby("exchange").agg
        (        
            trade_time_min = ("trade_time", 'min'),
            trade_time_latest = ("trade_time", 'max'),
            last_price_mean = ("last_price", 'mean'),
            last_vol_mean = ("last_vol", 'mean'),
            spread_mean = ("spread", 'mean'),
            num_trades = ("last_price", 'count')
        )
    )
    
    # Get time interval over which statistics have been calculated    
    df_agg["trade_time_duration"] = df_agg["trade_time_latest"] - df_agg["trade_time_min"]
    
    # Reset columns so that we can access exchanges below
    df_agg = df_agg.reset_index()
    
    # Calculate % of total volume for all exchanges
    last_vol_pert = []
    for i, row in df_agg.iterrows():
        try:
            df_vol = get_vol_exchange(row["exchange"], 30, base_curr)
            current_vol = df_vol["volume_SMA"][0]
            vol_pert = (row["last_vol_mean"] / current_vol) * 100
            last_vol_pert.append(vol_pert)
        except:
            last_vol_pert.append("")
            continue
            
    # Add % of total volume column
    df_agg["last_vol_pert"] = last_vol_pert
    
    # Remove redundant column
    df_agg = df_agg.drop(columns = ["trade_time_min"])
    
    # Round all float values
    # (seems to be overwritten by style below)
    df_agg = df_agg.round({"last_price_mean": 2,
                           "last_vol_mean": 2,
                           "spread_mean": 2
                          })
    
    display(df_agg.style.apply(highlight_max_min,
                               color = 'green',
                               subset = "last_price_mean")
           )
    
    return None

In [357]:
def highlight_max_min(x, color):
    
    return np.where((x == np.nanmax(x.to_numpy())) |
                    (x == np.nanmin(x.to_numpy())),
                    f"color: {color};",
                    None)

In [359]:
#display_agg_per_exchange(df_ex_all)

## Run bot

In [403]:
def run_bot(country,
            base_curr,
            target_curr):
    
    df_ex_all = get_trade_exchange_per_country(country, base_curr, target_curr)
    
    # Collect data every minute    
    while True:
        time.sleep(60)
        df_new = get_trade_exchange_per_country(country, base_curr, target_curr)
        
        # Merge to existing DataFrame
        df_ex_all = pd.concat([df_ex_all, df_new])
        
        # Remove duplicate rows based on all columns
        df_ex_all = df_ex_all.drop_duplicates()
        
        # Clear previous display once new one is available
        clear_output(wait = True)
        display_agg_per_exchange(df_ex_all, base_curr)        
        
    return None        

In [ ]:
run_bot("United States", "BTC", "USD")

/tmp/ipykernel_25321/3109473306.py:16: UserWarning: No data found for BTC-USD pair in tapbit
  warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
/tmp/ipykernel_25321/3109473306.py:16: UserWarning: No data found for BTC-USD pair in binance_us
  warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
/tmp/ipykernel_25321/3109473306.py:16: UserWarning: No data found for BTC-USD pair in metalx
  warnings.warn(f"No data found for {base_curr}-{target_curr} pair in {id}")
